In [27]:
import bs4 as bs
import requests
import json
from tqdm import tqdm
import re
from time import sleep

import aiohttp
import asyncio

In [2]:
CONSULTANT_PLUS_LINK = 'https://www.consultant.ru'

In [4]:
def filter_condition(text_str: str) -> bool:
    if 'Утратил силу' not in text_str \
            and 'в ред. Федерал' not in text_str \
            and 'текст в предыдущей редакции' not in text_str \
            and text_str:
        return True
    return False

In [40]:
laws = dict()
with open('./links.txt', 'r') as f:
    for link in tqdm(f):
        sleep(0.25)
        r = requests.get(f'{CONSULTANT_PLUS_LINK}/{link.strip()}')
        soup = bs.BeautifulSoup(r.text, 'html.parser')
        law_div = soup.find('div', class_='document-page__content document-page_left-padding')
        law_desc = law_div.find_all('p')
        law_desc_list = list(filter(filter_condition, [x.text for x in law_desc]))
        law_num, law_desc_str = '.'.join(law_desc_list[0].split('.')[:2]), ' '.join(law_desc_list[1:]).strip()
        laws[law_num] = list(
                            filter(
                                lambda x: x,
                                re.split(r'\d+\. ', law_desc_str)
                            )
                        )

788it [10:36,  1.24it/s]


In [41]:
laws

{'КоАП РФ Статья 5.1': ['Нарушение права гражданина на ознакомление со списком избирателей, участников референдума, либо нерассмотрение в установленный законом срок заявления о неправильности в списке избирателей, участников референдума, либо отказ выдать гражданину письменный ответ о причине отклонения заявления о внесении исправления в список избирателей, участников референдума - влечет наложение административного штрафа в размере от одной тысячи до одной тысячи пятисот рублей.'],
 'КоАП РФ Статья 5.3': ['Неисполнение решения избирательной комиссии, комиссии референдума, принятого в пределах ее компетенции, - влечет наложение административного штрафа на должностных лиц в размере от одной тысячи до двух тысяч рублей; на юридических лиц - от десяти тысяч до двадцати тысяч рублей. ',
  'Непредставление государственными органами, органами местного самоуправления, общественными объединениями, организациями независимо от формы собственности, в том числе организациями, осуществляющими теле-

In [42]:
with open('laws.json', 'w+', encoding='cp1251') as f:
    json.dump(laws, f, ensure_ascii=False)

# Эксперименты

In [31]:
r = requests.get('https://www.consultant.ru/document/cons_doc_LAW_34661/d152e9fe28b2a02960701035b5504cb6bdabc607/')

soup = bs.BeautifulSoup(r.text, 'html.parser')

In [32]:
law_div = soup.find('div', class_='document-page__content document-page_left-padding')
law_desc = law_div.find_all('p')

In [33]:
def filter_condition(text_str: str) -> bool:
    if 'Утратил силу' not in text_str \
            and 'в ред. Федерал' not in text_str \
            and 'текст в предыдущей редакции' not in text_str \
            and text_str:
        return True
    return False

In [34]:
law_desc_list = list(filter(filter_condition, [x.text for x in law_desc]))
law_num, law_desc_str = '.'.join(law_desc_list[0].split('.')[:2]), ' '.join(law_desc_list[1:]).strip()

In [35]:
law_desc_str

'1. Неисполнение решения избирательной комиссии, комиссии референдума, принятого в пределах ее компетенции, - влечет наложение административного штрафа на должностных лиц в размере от одной тысячи до двух тысяч рублей; на юридических лиц - от десяти тысяч до двадцати тысяч рублей. 2. Непредставление государственными органами, органами местного самоуправления, общественными объединениями, организациями независимо от формы собственности, в том числе организациями, осуществляющими теле- и (или) радиовещание, редакциями периодических печатных изданий, сетевых изданий, а также должностными лицами указанных органов и организаций в избирательную комиссию, комиссию референдума сведений и материалов, запрашиваемых комиссией в соответствии с законом, либо представление таких сведений и материалов с нарушением установленного законом срока, за исключением случаев, предусмотренных статьей 5.4, частью 1 статьи 5.17 и статьей 5.64 настоящего Кодекса, - влечет наложение административного штрафа на дол

In [39]:
list(
    filter(
        lambda x: x,
        re.split(r'\d+\. ', law_desc_str)
    )
)

['Неисполнение решения избирательной комиссии, комиссии референдума, принятого в пределах ее компетенции, - влечет наложение административного штрафа на должностных лиц в размере от одной тысячи до двух тысяч рублей; на юридических лиц - от десяти тысяч до двадцати тысяч рублей. ',
 'Непредставление государственными органами, органами местного самоуправления, общественными объединениями, организациями независимо от формы собственности, в том числе организациями, осуществляющими теле- и (или) радиовещание, редакциями периодических печатных изданий, сетевых изданий, а также должностными лицами указанных органов и организаций в избирательную комиссию, комиссию референдума сведений и материалов, запрашиваемых комиссией в соответствии с законом, либо представление таких сведений и материалов с нарушением установленного законом срока, за исключением случаев, предусмотренных статьей 5.4, частью 1 статьи 5.17 и статьей 5.64 настоящего Кодекса, - влечет наложение административного штрафа на дол